# How *spock* works

General note: If you wish to update the number of hours observe on the TRAPPIST telescopes as well you need to have acces to the **Hours_observation_TS_TN** google sheet. For this contact Elsa Ducrot (educrot@uliege.be)

## 1/ How `spock.long_term_scheduler` works

The SPECULOOS target list was contructed as described in [*Sebastian et al. 2020*](http://arxiv.org/abs/2011.02069) and include 1657 photometrically classified late-type dwarfs. These targets are divided in three distinct programs:
* Program 1: 365 dwarfs that are small and nearby enough to make possible the detailed atmospheric characterisation of an "Earth-like" planet with the upcoming JWST
* Program 2: 171 dwarfs of M5-type and later for which a significant detection of a planet similar to TRAPPIST-1b should be within reach of TESS
* Program 3: 1121 dwarfs later than M6-type that aims to perform a statistical census of short-period planets around ultracool dwarf stars.

## 2/ How `spock.short_term_scheduler` works

The aim of the `short_term_scheduler` is to modify existing plan by inserting a new block (corresponding to a *special* or *follow-up* target). Almost every night 1 ou 2 targets are observed per telescope. In that regards, the existing night block can be either situation A or situation B, see below:

![situationsAB](../../../_static/css/situations_AB.png)

### Situation A

If we consider we are in situation A, four different modifications can happend when the new block is inserted. 

* First, the new block can be scheduled in between the existing blocks, in this case the `end_time` of the first target and `start_time` of the second target are modified.

![situation_A1](../../../_static/css/situation_A1.png)

* Second, the new block can be scheduled on the first target's slot (from sun set), in this case only the `start_time` of the first target is modified.

![situationA2](../../../_static/css/situation_A2.png)

* Third, the new block can be scheduled on the second target's slot (up to sun rise), in this case only the `end_time` of the second target is modified.

![situationA3](../../../_static/css/situation_A3.png)

* Second, the new block is scheduled in the middel of an existing block, in this case the target is splited in two distinct blocks:
    * The first one (named target as the original block) with `start_time = old_start_time` and `end_time = start_time_new_block` 
    * The second one (named target_2) `start_time = end_time_new_block` and `end_time = old_end_time` 

![situationA4](../../../_static/css/situation_A4b.png)

### Situation B

If we consider we are in situation B, three different modifications can happend when the new block is inserted. 

* First, the new block can be scheduled in the second part of the night (up to sun rise), in this case only the `end_time` is modified.

![title](../../../_static/css/situation_B1.png)

* Second, the new block is scheduled in the smiddel of the night, in this case the target is splited in two distinct blocks:
    * The first one (named target as the original block)  with `start_time = old_start_time` and `end_time = start_time_new_block` 
    * The second one (named target_2) `start_time = end_time_new_block` and `end_time = old_end_time` 

![situationB2](../../../_static/css/situation_B2.png)

* Third, the new block is scheduled in the first part of the night (from sun set), in this case only the `start_time` is modified.

![situationB3](../../../_static/css/situation_B3.png)